In [ ]:
Item-Item based 
because users..
https://en.m.wikipedia.org/wiki/Collaborative_filtering

NN analyticsvidhya.com/blog/2021/05/movie-recommendations-using-keras-recommender-systems/
https://keras.io/examples/structured_data/collaborative_filtering_movielens/

Behaviour sequence transformer
https://keras.io/examples/structured_data/movielens_recommendations_transformers/

Do Users as well


"I’m going to go with item-item because it’s generally been found to be more effective, but also because it can immediately make fairly good recommendations to new users (assuming you already have other user reviews in place)."
https://medium.com/@jmcneilkeller/item-item-recommendation-with-surprise-4bf365355d96

In [ ]:
memory based
2 methods pearson correlation 
or cosine similartity
Nearest neighbours, 
disadvantages:as matrices become sparse, performance decreases - explain metrics obtained

model based
SVD - dimensionality reduction 
- better with sparsenes

In [2]:
import pandas as pd
import numpy as np

# KNN
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# Truncated SVD
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import surprise

In [3]:
# load data
pattern_details = pd.read_csv('data/consolidated_patterns.csv', low_memory=False)
df = pd.read_csv('data/less_sparse_users_patterns.csv', low_memory=False)

In [10]:
pattern_details.shape, df.shape

((188158, 24), (3250978, 5))

In [6]:
#  merge with df to get metadata for each pattern 
metadata = pattern_details[pattern_details['pattern_id'].isin(df.pattern_ids.values)]
# REINDEX??
print(metadata.shape)

(2963, 24)


### K-Nearest Neighbours

#### TODO - try different metrics (Euclidean, cosine, pearson, jaccard)

In [15]:
# from sklearn.neighbors import NearestNeighbors
# https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm 
# (to see which algorithm to use)
# # ‘ball_tree’ will use BallTree
# ‘kd_tree’ will use KDTree
# ‘brute’ will use a brute-force search.
# ‘auto’ will attempt to decide the most appropriate algorithm based
# on the values passed to fit method.
# 
 # default default=’minkowski’ (real value space) - try “jaccard” (boolean vector space)

In [14]:
user_knit = df
# user_knit = user_knit.drop_duplicates(['user_id', 'pattern_ids'])
user_knit_pivot = user_knit.pivot(index='pattern_ids', columns='user_id', values='has_knit').fillna(0)
user_knit_matrix = csr_matrix(user_knit_pivot.values)
user_knit_pivot.shape, user_knit_matrix.shape

In [17]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')  # default default=’minkowski’ (real value space) - try “jaccard” (boolean vector space)
model_knn.fit(user_knit_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [18]:
# give recommendation - need to reference this article!! 
query_index = np.random.choice(user_knit_pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(user_knit_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 10)

1600


In [19]:
# TODO turn into one function! pattern id to name! 

def get_index_from_pattern_id(pattern_id):
    return metadata[metadata.pattern_id == pattern_id].index[0]

pattern_id = int(user_knit_pivot.index[query_index])
pattern_index = get_index_from_pattern_id(pattern_id)

def get_pattern_name_from_index(pattern_index):
    return metadata[metadata.index == pattern_index]["name_permalink"].values[0]

In [21]:
# give recomendations for the pattern selected
for i in range(0, len(distances.flatten())):
    if i == 0:
#         print('Recommendations for {0} :\n'.format(user_knit_pivot.index[query_index]))
        print('Recommendations for {0} :\n'.format(get_pattern_name_from_index(get_index_from_pattern_id(int(user_knit_pivot.index[query_index])))))
    else:
        pattern_id = int(user_knit_pivot.index[indices.flatten()[i]])
        pattern_metadata_index = get_index_from_pattern_id(pattern_id)
        pattern_name = get_pattern_name_from_index(pattern_metadata_index)
 
        print('{0}: {1} {2}, with distance of: {3}'
                .format(i, pattern_id, pattern_name, distances.flatten()[i]))
    

Recommendations for bandana-cowl :

1: 169260 honey-cowl, with distance of: 0.8732878156686384
2: 426231 barley-2, with distance of: 0.8759726749468475
3: 181549 the-age-of-brass-and-steam-kerchief, with distance of: 0.8854555800880314
4: 211562 hitchhiker, with distance of: 0.8878690389677955
5: 226359 rikke-hat, with distance of: 0.8905057869660706
6: 75857 turn-a-square, with distance of: 0.8925989179925576
7: 464893 garter-ear-flap-hat, with distance of: 0.8997002369473893
8: 124400 sockhead-slouch-hat, with distance of: 0.8998918881352115
9: 216488 gap-tastic-cowl, with distance of: 0.9015507120664095


### Matrix factorization 
#### a) Truncated SVD (explain all of the methods)

In [ ]:
user_knit = df
user_knit_pivot = user_knit.pivot(index='pattern_ids', columns='user_id', values='has_knit').fillna(0)
X = user_knit_pivot.values

In [ ]:
svd = TruncatedSVD(n_components=1000, random_state =88)
matrix = svd.fit_transform(X)
matrix.shape

In [ ]:
# To determine best nymber (have increased from 10 iteratively to 1000 (explained variab
# should be above 0.9, but 

explained_variance_ratios = svd.explained_variance_ratio_
explained_variance_ratios_sum = svd.explained_variance_ratio_.sum()
singular_values = svd.singular_values_
print(explained_variance_ratios_sum )

In [ ]:
## PLOT EXPLAINED VARIANCE OVER COMPONENTS

In [ ]:
# calculate pearson R correlation 
corr = np.corrcoef(matrix)
corr.shape

In [22]:
# get pattern id's out 
pattern_ids = user_knit_pivot.index
pattern_ids_list = list(pattern_ids)

In [ ]:
# Find recommendations for a particular pattern:
pattern_100315 = pattern_ids_list.index('100315')
corr_100315 = corr[pattern_100315]
list(pattern_ids[(corr_100315<1.0)&(corr_100315>0.95)])

In [ ]:
#MAKE FUNCTION!! HOW TO CROSS VALIDATE??

In [ ]:
# plot with Tsne?? (https://towardsdatascience.com/recommender-system-singular-value-decomposition-svd-truncated-svd-97096338f361)

# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# %matplotlib inline

# tsne = TSNE(perplexity=5, n_components=2, init="pca", n_iter=5000)
# plot_only = 50
# coords = tsne.fit_transform(movie_features[:plot_only, :])

# plt.figure(figsize=(18, 18))
# labels = [df_mv4.iloc[i].title for i in range(plot_only)]
# for i, label in enumerate(labels):
#     x, y = coords[i, :]
#     plt.scatter(x, y)
#     plt.annotate(label,
#                  xy=(x, y),
#                  xytext=(10, 4),
#                  textcoords="offset points",
#                  ha="right",
#                  va="bottom")

# plt.show()

### Straight SVD no pp or surprise etc... 
https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/

In [23]:
user_knit = df
user_knit_pivot = user_knit.pivot(index='pattern_ids', columns='user_id', values='has_knit').fillna(0)
X = user_knit_pivot.values

In [ ]:
A = X.T / np.sqrt(X.shape[0] - 1)
U, S, V = np.linalg.svd(A)

In [ ]:
# Most similar
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

In [ ]:
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

In [ ]:
#k-principal components to represent movies, movie_id to find recommendations, top_n print n results        
k = 50
movie_id = 10 # (getting an id from movies.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)

#Printing the top N similar movies
print_similar_movies(movie_data, movie_id, indexes)

### Using the implicit library
Using Surprise Library 
(doesn't suport implicit ratings, but the SVDpp algoithm  library does), nor item item??
https://implicit.readthedocs.io/en/latest/quickstart.html
https://towardsdatascience.com/building-a-collaborative-filtering-recommender-system-with-clickstream-data-dffc86c8c65

** Interesting REad
https://nbviewer.org/github/jmsteinw/Notebooks/blob/master/RecEngine_NB.ipynb

In [ ]:
# not going to tune as prone to overfitting https://towardsdatascience.com/svd-where-model-tuning-goes-wrong-61c269402919
https://towardsdatascience.com/building-a-collaborative-filtering-recommender-system-with-clickstream-data-dffc86c8c65

In [2]:
import implicit
import numpy as np
import pandas as pd
import scipy.sparse as sparse

In [3]:
pattern_details = pd.read_csv('data/consolidated_patterns.csv', low_memory=False)
df = pd.read_csv('data/less_sparse_users_patterns.csv', low_memory=False)

In [6]:
df.columns
dataset = df[['user_id', 'pattern_ids', 'has_knit']]
dataset.columns = ['personId', 'contentId', 'has_knit']

# item_user_data= df[['user_id', 'pattern_ids']]

# user_knit = df
# # user_knit = user_knit.drop_duplicates(['user_id', 'pattern_ids'])
# user_knit_pivot = user_knit.pivot(index='pattern_ids', columns='user_id', values='has_knit').fillna(0)
# user_knit_matrix = csr_matrix(user_knit_pivot.values)
# user_knit_pivot.shape, user_knit_matrix.shape

In [8]:
# grouped_df['title'] = grouped_df['title'].astype("category")
dataset['personId'] = dataset['personId'].astype("category")
dataset['contentId'] = dataset['contentId'].astype("category")
dataset['person_id'] = dataset['personId'].cat.codes
dataset['content_id'] = dataset['contentId'].cat.codes

sparse_content_person = sparse.csr_matrix((dataset['has_knit'].astype(float), (dataset['content_id'], dataset['person_id'])))
sparse_person_content = sparse.csr_matrix((dataset['has_knit'].astype(float), (dataset['person_id'], dataset['content_id'])))

# model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

# alpha = 15
# data = (sparse_content_person * alpha).astype('double')
# model.fit(data)

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

alpha = 15
data = (sparse_content_person * alpha).astype('double')
model.fit(data)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(user_knit_matrix )



  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
content_id = 450
n_similar = 10

person_vecs = model.user_factors
content_vecs = model.item_factors

content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1))

scores = content_vecs.dot(content_vecs[content_id]) / content_norms
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / content_norms[content_id]), key=lambda x: -x[1])

for content in similar:
    idx, score = content
    print(grouped_df.title.loc[grouped_df.content_id == idx].iloc[0])

In [ ]:
# recommend items for a user
user_items = item_user_data.T.tocsr()
recommendations = model.recommend(userid, user_items)

# find related items
related = model.similar_items(itemid)

In [9]:
dataset.head()

,uid,iid,rating
0,5153610,88,1
1,511279,88,1
2,3146767,88,1
3,21965,88,1
4,55722,88,1


In [ ]:
# TO DO TEST TRAIN SPLIT AND assess with area under the curve like in 
# https://nbviewer.org/github/jmsteinw/Notebooks/blob/master/RecEngine_NB.ipynb

explain how it works 